# Этап 1: Подготовка данных

#### Загрузка данных

In [ ]:
# Импорт стандартных библиотек
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv('sp500_historical_data.csv')

#### Преобразование столбца даты

In [ ]:
data['Date'] = pd.to_datetime(data['Date'])

#### Удаление данных за выходные дни

In [ ]:
data = data[data['Date'].dt.weekday < 5]

#### Определение полного диапазона дат

In [ ]:
full_date_range = pd.date_range(start=data['Date'].min(), end=data['Date'].max(), freq='B')  # 'B' обозначает бизнес-день

#### Проверка наличия данных по каждому тикеру

In [ ]:
data_pivot = data.pivot_table(index='Date', columns='Ticker', values='Adj Close')
missing_percent = data_pivot.isnull().mean() * 100
tickers_to_remove = missing_percent[missing_percent > 5].index.tolist()
clean_data = data[~data['Ticker'].isin(tickers_to_remove)]

#### Проверка на дупликаты

In [ ]:
clean_data = clean_data.drop_duplicates()

#### Обработка подозрительно низкого объема торгов

In [ ]:
volume_threshold = clean_data['Volume'].quantile(0.01)
low_volume_data = clean_data[clean_data['Volume'] <= volume_threshold]
low_volume_data.to_csv('low_volume_records.csv', index=False)

#### Проверка размерности данных

In [ ]:
data_pivot_clean = clean_data.pivot_table(index='Date', columns='Ticker', values='Adj Close')
missing_dates_per_ticker = data_pivot_clean.isnull().sum()
tickers_with_missing_dates = missing_dates_per_ticker[missing_dates_per_ticker > 0].index.tolist()
clean_data = clean_data[~clean_data['Ticker'].isin(tickers_with_missing_dates)]

#### Сохранение очищенных данных

In [ ]:
clean_tickers = clean_data['Ticker'].unique()
clean_data.to_csv('cleaned_sp500_data.csv', index=False)

In [ ]:
import os

# Для разделения данных и предобработки
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Для построения модели LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Для генетического алгоритма
from deap import base, creator, tools, algorithms

# Для оценки модели
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Для визуализации
import matplotlib.pyplot as plt
%matplotlib inline

# Дополнительные библиотеки
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore")

#### Сбор информации о секторах

In [ ]:
# Установка yfinance при необходимости
# !pip install yfinance

import yfinance as yf

# Получение списка уникальных тикеров
tickers = data['Ticker'].unique()

# Инициализация словаря для хранения информации о секторах
ticker_sectors = {}

# Получение информации о секторах для каждого тикера
for ticker in tickers:
    try:
        stock_info = yf.Ticker(ticker).info
        sector = stock_info.get('sector', 'Unknown')
        ticker_sectors[ticker] = sector
    except Exception as e:
        print(f"Не удалось получить данные для {ticker}: {e}")
        ticker_sectors[ticker] = 'Unknown'

# Создание DataFrame из словаря
sectors_df = pd.DataFrame(list(ticker_sectors.items()), columns=['Ticker', 'Sector'])

# Объединение информации о секторах с основными данными
data = pd.merge(data, sectors_df, on='Ticker', how='left')

#### Загрузка очищенных данных

In [ ]:
# Загрузка очищенных данных
data = pd.read_csv('cleaned_sp500_data.csv')

# Преобразование столбца 'Date' в формат datetime
data['Date'] = pd.to_datetime(data['Date'])

# Этап 2: Построение и оптимизация модели LSTM с помощью GA

#### Подготовка данных

In [27]:
# Определение параметров
TIME_STEPS = 30  # Длина последовательности для входа в LSTM

# Инициализация словарей для хранения результатов
models = {}
scalers = {}
volatility = {}
performance = {}

#### Определение функций для подготовки данных

In [29]:
def create_sequences(data, time_steps=TIME_STEPS):
    X = []
    y = []
    for i in range(len(data) - time_steps):
        X.append(data[i:(i + time_steps)])
        y.append(data[i + time_steps])
    return np.array(X), np.array(y)

#### Определение функции оценки модели

In [32]:
import random
import tensorflow.keras.backend as K

def evaluate_model(individual, X_train, y_train, X_val, y_val):
    # Распаковка гиперпараметров из индивидуального решения
    n_layers = individual[0]
    n_neurons = individual[1]
    dropout_rate = individual[2]
    learning_rate = individual[3]

    # Построение модели
    K.clear_session()  # Очистка предыдущих моделей из памяти
    model = Sequential()

    if n_layers == 1:
        # Единственный LSTM слой
        model.add(LSTM(units=n_neurons, return_sequences=False, input_shape=(X_train.shape[1], X_train.shape[2])))
        model.add(Dropout(dropout_rate))
    else:
        # Первый LSTM слой
        model.add(LSTM(units=n_neurons, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
        model.add(Dropout(dropout_rate))
        # Средние LSTM слои
        for _ in range(n_layers - 2):
            model.add(LSTM(units=n_neurons, return_sequences=True))
            model.add(Dropout(dropout_rate))
        # Последний LSTM слой
        model.add(LSTM(units=n_neurons, return_sequences=False))
        model.add(Dropout(dropout_rate))

    model.add(Dense(1))

    # Компиляция модели
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(loss='mean_squared_error', optimizer=optimizer)

    # Обучение модели
    history = model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_val, y_val), verbose=0)

    # Оценка модели на валидационных данных
    val_loss = history.history['val_loss'][-1]

    return val_loss,

#### Определение процесса генетического алгоритма

In [34]:
def custom_mutation(individual, indpb):
    for i in range(len(individual)):
        if random.random() < indpb:
            if i == 0:  # n_layers (целое число)
                individual[i] = random.randint(1, 2)  # Максимум 2 слоя
            elif i == 1:  # n_neurons (целое число)
                individual[i] = random.randint(32, 128)  # От 32 до 128 нейронов
            elif i == 2:  # dropout_rate (вещественное число)
                individual[i] = random.uniform(0.0, 0.3)  # От 0.0 до 0.3
            elif i == 3:  # learning_rate (вещественное число)
                individual[i] = random.uniform(0.0005, 0.005)  # Уменьшенный диапазон
    return individual,

In [35]:
from joblib import Parallel, delayed

def optimize_model_with_ga(X_train, y_train, X_val, y_val, n_generations, population_size):
    # Определение индивидуала и функции приспособленности
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))  # Минимизируем валидационную ошибку
    creator.create("Individual", list, fitness=creator.FitnessMin)

    toolbox = base.Toolbox()

    # Определение атрибутов для каждого гиперпараметра
    toolbox.register("attr_n_layers", random.randint, 1, 3)
    toolbox.register("attr_n_neurons", random.randint, 32, 256)
    toolbox.register("attr_dropout_rate", random.uniform, 0.0, 0.5)
    toolbox.register("attr_learning_rate", random.uniform, 0.0001, 0.01)

    # Создание индивидуала
    toolbox.register("individual", tools.initCycle, creator.Individual,
                     (toolbox.attr_n_layers, toolbox.attr_n_neurons, toolbox.attr_dropout_rate, toolbox.attr_learning_rate), n=1)

    # Создание популяции
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    
    # Определение функции оценки
    def eval_function(individual):
        return evaluate_model(individual, X_train, y_train, X_val, y_val)

    # Регистрация функций
    toolbox.register("evaluate", eval_function)
    toolbox.register("mate", tools.cxUniform, indpb=0.5)
    toolbox.register("mutate", custom_mutation, indpb=0.2)
    toolbox.register("select", tools.selTournament, tournsize=3)

    # Инициализация популяции
    pop = toolbox.population(n=population_size)

    # Запуск генетического алгоритма
    hof = tools.HallOfFame(1)  # Отслеживаем лучшее решение

    algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=n_generations, halloffame=hof, verbose=False)

    best_individual = hof[0]
    return best_individual

#### Обучение модели для каждого тикера

In [ ]:
# Создание каталогов для сохранения моделей и графиков
if not os.path.exists('models'):
    os.makedirs('models')
if not os.path.exists('plots'):
    os.makedirs('plots')

# Цикл по каждому тикеру
for ticker in tickers:
    if not os.path.isfile(f'models/{ticker}_model.keras'):
        print(f"Обработка тикера: {ticker}")

        # Получение данных для тикера
        df_ticker = data[data['Ticker'] == ticker].sort_values('Date')

        # Используем 'Adj Close' в качестве целевой переменной
        close_prices = df_ticker['Adj Close'].values.reshape(-1, 1)

        # Проверка достаточности данных
        if len(close_prices) <= TIME_STEPS:
            print(f"Недостаточно данных для тикера {ticker}")
            continue

        # Масштабирование данных
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled_data = scaler.fit_transform(close_prices)
        scalers[ticker] = scaler

        # Создание последовательностей
        X, y = create_sequences(scaled_data)

        # Проверка достаточности последовательностей
        if len(X) == 0:
            print(f"Недостаточно последовательностей для тикера {ticker}")
            continue

        # Разделение данных на обучающую и тестовую выборки (80% на обучение)
        split_index = int(0.8 * len(X))
        X_train_full, X_test = X[:split_index], X[split_index:]
        y_train_full, y_test = y[:split_index], y[split_index:]

        # Дополнительное разделение обучающих данных на обучение и валидацию (80% на обучение)
        val_split_index = int(0.8 * len(X_train_full))
        X_train, X_val = X_train_full[:val_split_index], X_train_full[val_split_index:]
        y_train, y_val = y_train_full[:val_split_index], y_train_full[val_split_index:]

        # Оптимизация модели с помощью GA
        best_hyperparams = optimize_model_with_ga(X_train, y_train, X_val, y_val, n_generations=3, population_size=5)

        # Извлечение лучших гиперпараметров
        n_layers = best_hyperparams[0]
        n_neurons = best_hyperparams[1]
        dropout_rate = best_hyperparams[2]
        learning_rate = best_hyperparams[3]

        # Построение финальной модели с лучшими гиперпараметрами
        K.clear_session()
        model = Sequential()

        if n_layers == 1:
            # Единственный LSTM слой
            model.add(LSTM(units=n_neurons, return_sequences=False, input_shape=(X_train_full.shape[1], X_train_full.shape[2])))
            model.add(Dropout(dropout_rate))
        else:
            # Первый LSTM слой
            model.add(LSTM(units=n_neurons, return_sequences=True, input_shape=(X_train_full.shape[1], X_train_full.shape[2])))
            model.add(Dropout(dropout_rate))
            # Средние LSTM слои
            for _ in range(n_layers - 2):
                model.add(LSTM(units=n_neurons, return_sequences=True))
                model.add(Dropout(dropout_rate))
            # Последний LSTM слой
            model.add(LSTM(units=n_neurons, return_sequences=False))
            model.add(Dropout(dropout_rate))

        model.add(Dense(1))

        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
        model.compile(loss='mean_squared_error', optimizer=optimizer)

        # Обучение финальной модели на полной обучающей выборке
        model.fit(X_train_full, y_train_full, epochs=20, batch_size=32, verbose=0)

        # Оценка модели на тестовых данных
        y_pred = model.predict(X_test)
        y_pred_inverse = scaler.inverse_transform(y_pred)
        y_test_inverse = scaler.inverse_transform(y_test.reshape(-1, 1))

        # Расчет метрик оценки
        mse = mean_squared_error(y_test_inverse, y_pred_inverse)
        mae = mean_absolute_error(y_test_inverse, y_pred_inverse)

        performance[ticker] = {'MSE': mse, 'MAE': mae}

        # Сохранение модели
        model.save(f'models/{ticker}_model.keras')
        models[ticker] = model

        # Визуализация результатов
        plt.figure(figsize=(10, 6))
        plt.plot(y_test_inverse, label='Реальная цена')
        plt.plot(y_pred_inverse, label='Предсказанная цена')
        plt.title(f'Предсказание цены для {ticker}')
        plt.xlabel('Время')
        plt.ylabel('Цена')
        plt.legend()
        plt.savefig(f'plots/{ticker}_prediction.png')
        plt.close()
    else:
        print(f"Модель для тикера: {ticker} уже создана")

Модель для тикера: MMM уже создана
Модель для тикера: AOS уже создана
Модель для тикера: ABT уже создана
Модель для тикера: ABBV уже создана
Модель для тикера: ACN уже создана
Модель для тикера: ADBE уже создана
Модель для тикера: AMD уже создана
Модель для тикера: AES уже создана
Модель для тикера: AFL уже создана
Модель для тикера: A уже создана
Модель для тикера: APD уже создана
Модель для тикера: AKAM уже создана
Модель для тикера: ALB уже создана
Модель для тикера: ARE уже создана
Модель для тикера: ALGN уже создана
Модель для тикера: ALLE уже создана
Модель для тикера: LNT уже создана
Модель для тикера: ALL уже создана
Модель для тикера: GOOGL уже создана
Модель для тикера: GOOG уже создана
Модель для тикера: MO уже создана
Модель для тикера: AMZN уже создана
Модель для тикера: AMCR уже создана
Модель для тикера: AEE уже создана
Модель для тикера: AEP уже создана
Модель для тикера: AXP уже создана
Модель для тикера: AIG уже создана
Модель для тикера: AMT уже создана
Модель для ти

Exception ignored in: <function WeakKeyDictionary.__init__.<locals>.remove at 0x335ab0220>
Traceback (most recent call last):
  File "/Users/kate/anaconda3/lib/python3.11/weakref.py", line 369, in remove
    def remove(k, selfref=ref(self)):

KeyboardInterrupt: 


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Обработка тикера: DVA


Exception ignored in: <function WeakKeyDictionary.__init__.<locals>.remove at 0x335ab0220>
Traceback (most recent call last):
  File "/Users/kate/anaconda3/lib/python3.11/weakref.py", line 369, in remove
    def remove(k, selfref=ref(self)):

KeyboardInterrupt: 


#### Сохранение волатильности и метрик производительности

In [ ]:
for ticker in tickers:
    # Расчет доходностей для оценки волатильности
    returns = np.log(close_prices[1:] / close_prices[:-1])

    # Оценка волатильности (стандартное отклонение доходностей)
    volatility_value = np.std(returns)
    volatility[ticker] = volatility_value
    
# Преобразование словаря волатильности в DataFrame
volatility_df = pd.DataFrame.from_dict(volatility, orient='index', columns=['Volatility'])
volatility_df.to_csv('volatility.csv')

# Преобразование словаря производительности в DataFrame
performance_df = pd.DataFrame.from_dict(performance, orient='index')
performance_df.to_csv('model_performance.csv')

#### Организация и документирование результатов

In [ ]:
# Создание каталогов при необходимости (если не были созданы ранее)
if not os.path.exists('models'):
    os.makedirs('models')
if not os.path.exists('plots'):
    os.makedirs('plots')